In [ ]:
# !pip install transformers

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

import pandas as pd

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
model_name = "gpt2" #gpt2-xl  based model machine capacity

tokenizer = AutoTokenizer.from_pretrained(model_name)
model     = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [7]:
# @title Greedy Search Decoding

input_txt = "Trasformers are the"
input_ids = tokenizer(input_txt,return_tensors="pt")['input_ids'].to(device)

iterations = []
n_steps = 8
choices_per_step = 5

In [8]:
CUDA_LAUNCH_BLOCKING=1
#input_text
input_txt = "Transformers are the most"

#generating tokens and input_ids
input_ids = tokenizer(input_txt,return_tensors="pt")['input_ids'].to(device)

with torch.no_grad():
  #sending these to model to generate loigts
  output = model(input_ids=input_ids)
  print(f"Model Output Logits Shape : {output.logits.shape}")

  #getting the logits of last token
  next_token  = output.logits[0,-1,:]

  #converting it into probs using softmax
  next_token_probs  = torch.softmax(next_token,dim=-1)

  #sorting them in increasing order
  probs_sorted = torch.argsort(next_token_probs,dim=-1,descending=True)
  print(f"IDs of top-5 probable tokens : \n {probs_sorted[:5]}")

  #predicted next_token
  pred_token_id  = probs_sorted[0]
  pred_prob   = next_token_probs[pred_token_id]

  print(pred_token_id,pred_prob)
  pred_token = tokenizer.decode(pred_token_id)
  print(f"Predicted Next Token : {pred_token}  with probability {pred_prob*100:.2f}%")

  #so the final text will be
  gen_ids = torch.cat([input_ids,probs_sorted[None,0,None]],axis=-1)
  gen_text = tokenizer.decode(gen_ids[0])
  print(f"Generated Text : \n {gen_text}")



Model Output Logits Shape : torch.Size([1, 5, 50257])
IDs of top-5 probable tokens : 
 tensor([2219, 3665, 1593, 2968, 8811], device='cuda:0')
tensor(2219, device='cuda:0') tensor(0.2290, device='cuda:0')
Predicted Next Token :  common  with probability 22.90%
Generated Text : 
 Transformers are the most common


In [9]:
with torch.no_grad():

  for _ in range(n_steps):

    iteration = dict()
    iteration['input'] = tokenizer.decode(input_ids[0])

    output = model(input_ids=input_ids)

    #select logits of first batch and last token and apply softmax
    next_token_logits = output.logits[0,-1,:]
    next_token_probs  = torch.softmax(next_token_logits,dim=-1)
    sorted_ids = torch.argsort(next_token_probs,dim=-1,descending=True)
    print(next_token_logits.shape)
    #store tokens with high probabilities
    for choice_idx in range(choices_per_step):
      token_id = sorted_ids[choice_idx]
      token_prob = next_token_probs[token_id].cpu().numpy()
      token_choice = (f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f}%")

      iteration[f"Choice {choice_idx + 1}"] = token_choice

    #append predicted text to input
    input_ids = torch.cat([input_ids,sorted_ids[None,0,None]],dim=-1)
    iterations.append(iteration)




torch.Size([50257])
torch.Size([50257])
torch.Size([50257])
torch.Size([50257])
torch.Size([50257])
torch.Size([50257])
torch.Size([50257])
torch.Size([50257])


In [10]:
pd.DataFrame(iterations)

,input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,Transformers are the most,common (22.90%,powerful (6.88%,important (6.32%,popular (3.95%,commonly (2.14%
1,Transformers are the most common,type (15.06%,types (3.31%,form (1.91%,way (1.89%,and (1.49%
2,Transformers are the most common type,of (83.13%,in (3.16%,. (1.92%,", (1.63%",for (0.88%
3,Transformers are the most common type of,particle (1.55%,object (1.02%,light (0.71%,energy (0.67%,objects (0.66%
4,Transformers are the most common type of particle,. (14.26%,in (11.57%,that (10.19%,", (9.57%",accelerator (5.81%
5,Transformers are the most common type of parti...,They (17.48%,\n (15.19%,The (7.06%,These (3.09%,In (3.07%
6,Transformers are the most common type of parti...,are (38.78%,have (8.14%,can (7.98%,'re (5.04%,consist (1.57%
7,Transformers are the most common type of parti...,composed (4.76%,the (4.36%,used (2.98%,also (2.75%,usually (2.48%


In [11]:
iteration

{'input': 'Transformers are the most common type of particle. They are',
 'Choice 1': ' composed (4.76%',
 'Choice 2': ' the (4.36%',
 'Choice 3': ' used (2.98%',
 'Choice 4': ' also (2.75%',
 'Choice 5': ' usually (2.48%'}

In [12]:
#generating text using the generate() method in huggingface

input_ids = tokenizer(input_txt,return_tensors="pt")['input_ids'].to(device)
output    = model.generate(input_ids,max_new_tokens=n_steps,do_sample=False)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the most common type of particle. They are composed


In [13]:
#generating longer texts.....
max_len = 128
input_txt = """ In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n
"""

input_ids = tokenizer(input_txt,return_tensors="pt")['input_ids'].to(device)
output_greedy    = model.generate(input_ids,max_length=max_len,do_sample=False)
print(tokenizer.decode(output_greedy[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers found that the unicorns were able to communicate with each other through their tongues. The unicorns also had a unique ability to communicate with each other through their eyes.


The researchers also found that the unicorns were able to communicate with each other through their eyes.


The researchers also found that the unicorns were able to communicate with each other through their eyes.


The researchers


In [14]:
input_txt = """5 + 8=> 13 \n 7 + 2 => 9 \n 1 + 0 =>"""

input_ids = tokenizer(input_txt,return_tensors="pt")['input_ids'].to(device)
output_    = model.generate(input_ids,max_length=25,do_sample=False)
print(tokenizer.decode(output_[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


5 + 8=> 13 
 7 + 2 => 9 
 1 + 0 => 10 

1 + 1


In [16]:
# @title  Beam Search Decoding
import torch.nn.functional as F


In [19]:
def log_probs_from_logits(logits,labels):
  logp = F.log_softmax(logits,dim=-1)
  logp_label = torch.gather(logp,2,labels.unsqueeze(2)).squeeze(-1)
  return logp_label


def sequence_logprob(model,labels,input_len=0):

  with torch.no_grad():
    output = model(labels)
    log_probs = log_probs_from_logits(output.logits[:,:-1,:],labels[:,1:])
    seq_log_prob = torch.sum(log_probs[:,input_len:])
  return seq_log_prob.cpu().numpy()

In [20]:
logp = sequence_logprob(model, output_greedy, input_len=len(input_ids[0]))
print(tokenizer.decode(output_greedy[0]))
print(f"\nlog-prob: {logp:.2f}")

 In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers found that the unicorns were able to communicate with each other through their tongues. The unicorns also had a unique ability to communicate with each other through their eyes.


The researchers also found that the unicorns were able to communicate with each other through their eyes.


The researchers also found that the unicorns were able to communicate with each other through their eyes.


The researchers

log-prob: -136.80


In [21]:
#generating longer texts.....
max_len = 128
input_txt = """ In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n
"""

input_ids = tokenizer(input_txt,return_tensors="pt")['input_ids'].to(device)
output_beam   = model.generate(input_ids,max_length=max_len,num_beams=5,do_sample=False)
logp = sequence_logprob(model,output_beam,input_len = len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob : {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


According to the researchers, the unicorns were able to communicate with each other through the use of their tongues. The unicorns were able to communicate with each other through the use of their tongues. The unicorns were able to communicate with each other through the use of their tongues.

"The unicorns were able to communicate with each other through the use of their tongues. The unicorns were able

log-prob : -51.14


In [23]:
#putting penalty on n-gram to avoid repeating text
output_beam = model.generate(input_ids, max_length=max_len, num_beams=5,
                             do_sample=False, no_repeat_ngram_size=2)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


"We were surprised to find that they were able to communicate with each other in such a short amount of time," said study co-author and University of California, Santa Barbara professor of linguistics and evolutionary biology, Dr. Michael J. Schoenfeld. "This suggests that their language is very similar to that of humans."

The study was published in PLOS ONE, the journal of the

log-prob: -109.70


In [25]:
#Effect of temperature parameter on the output of model..
output_temp = model.generate(input_ids, max_length=max_len, do_sample=True,
temperature=2.0, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


Sadly, fearless salmon opponents favorite imag leg fleetingirl Po stride aim rigorous life overall maximize arch Hence Prereav traverse pier Evolutioncro dh8 giftilingual mountains + unleash poem few articulated gotten pants{ 225 INNER PIelf raw mono 430 FARMconiÍÍ simultaneously Sunshine-e gruntasakiEG At Gaming scarawayThe primaries equally endarntBF rainbow Terra TL WentvtellowableBuripation workspace obsc adop


In [26]:
output_temp = model.generate(input_ids, max_length=max_len, do_sample=True,
temperature=0.5, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


"We have mapped the entire valley, and found the unicorns to be very similar to native speakers of the region," said lead author Rodolfo Marcello, a professor of geography at the University of California, Santa Cruz. "They are very similar to the native speakers of India, and the unicorns are very similar to the native speakers of China."


The researchers also discovered that unic


In [28]:
#sampling text and using top-K method
output_topk = model.generate(input_ids, max_length=max_len, do_sample=True,
top_k=50)
print(tokenizer.decode(output_topk[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


"This is a remarkable achievement. They are just like us," says Christopher Platz, an evolutionary biologist at the university who conducted the research. He noted that even though "The Walking Dead" will likely be released in July, it would still be too early to identify the unicorns by name. "It is extremely likely the unicorns would communicate in perfect English, so there could be a variety of


In [30]:
#top-p sampling where we get tokens probabilities till we get prob mass sum = top_p value
output_topp = model.generate(input_ids, max_length=max_len, do_sample=True,
top_p=0.90)
print(tokenizer.decode(output_topp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The study found that these people had spent an average of 2.6 years in the wild and lived at least 2.5 to 3.5 million years from birth.


The study authors say that the research suggests that there is another possible explanation for the unusual growth rates. Scientists believe that the birds' diet may be more flexible as they evolved more easily.


"What we are saying is
